<a href="https://colab.research.google.com/github/Pataweepr/applyML_vistec_2019/blob/master/hw3_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Titanic

พวกเราจะพยายามให้ logictic reg เพื่อทำนาย แบบจำลอง

[kaggle](https://www.kaggle.com/c/titanic)

[linear_model](https://scikit-learn.org/stable/modules/linear_model.html)

[confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)

[link](https://stackoverflow.com/questions/49310470/using-kaggle-datasets-into-google-colab)

[seaborn](https://seaborn.pydata.org/)

In [0]:
# import library 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score,mean_squared_error
from google.colab import files
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from IPython.display import display
import json
import seaborn as sns

from scipy import stats

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
!unzip '/content/gdrive/My Drive/house-prices-advanced-regression-techniques.zip'
!ls

sample_submission = pd.read_csv('sample_submission.csv')
display(sample_submission.head())

In [0]:
train_data = pd.read_csv('train.csv')
train_data, val_data = train_test_split(train_data, test_size=1/11, random_state=30)
test_data = pd.read_csv('test.csv')

display(train_data.shape)
display(val_data.shape)
display(test_data.shape)

LotFrontage
GarageYrBlt
MasVnrArea

In [0]:
display(train_data.head())
display(val_data.head())
display(test_data.head())

#train_data.drop(columns =['Alley','MiscFeature'])

train_data["LotFrontage"] = train_data["LotFrontage"].fillna(train_data["LotFrontage"].mode().iloc[0])
train_data["GarageYrBlt"] = train_data["GarageYrBlt"].fillna(train_data["GarageYrBlt"].mode().iloc[0])
train_data["MasVnrArea"] = train_data["MasVnrArea"].fillna(train_data["MasVnrArea"].mode().iloc[0])

val_data["LotFrontage"] = val_data["LotFrontage"].fillna(train_data["LotFrontage"].mode().iloc[0])
val_data["GarageYrBlt"] = val_data["GarageYrBlt"].fillna(train_data["GarageYrBlt"].mode().iloc[0])
val_data["MasVnrArea"] = val_data["MasVnrArea"].fillna(train_data["MasVnrArea"].mode().iloc[0])

In [0]:
def get_feature_groups(ames_df):
    # Numerical Features
    num_features = ames_df.select_dtypes(include=['int64','float64']).columns
    num_features = num_features.drop(['Id','SalePrice']) # drop ID and SalePrice

    # Categorical Features
    cat_features = ames_df.select_dtypes(include=['object']).columns
    return list(num_features), list(cat_features)

In [0]:
num_features, cat_features = get_feature_groups(train_data)

# Grid of distribution plots of all numerical features
f = pd.melt(train_data, value_vars=sorted(num_features))
g = sns.FacetGrid(f, col='variable', col_wrap=4, sharex=False, sharey=False)
g = g.map(sns.distplot, 'value')

# Grid of frequency plots of all categoriccal features
f = pd.melt(train_data, value_vars=sorted(cat_features))
g = sns.FacetGrid(f, col='variable', col_wrap=4, sharex=False, sharey=False)
plt.xticks(rotation='vertical')
g = g.map(sns.countplot, 'value')
[plt.setp(ax.get_xticklabels(), rotation=60) for ax in g.axes.flat]
g.fig.tight_layout()
plt.show()

In [0]:
for col in train_data.columns:
  print(col , ': ',train_data[col].isnull().values.any())
print('------------------------------------------------------------------------------------------------------------')
for col in test_data.columns:
  print(col , ': ',test_data[col].isnull().values.any())

[stats.probplot](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.probplot.html)

In [0]:
price_train = np.array(train_data['SalePrice'].values)
n, bins, patches = plt.hist(price_train, 100, density=True, facecolor='g', alpha=0.75)
plt.show()


fig = plt.figure()
res = stats.probplot(train_data['SalePrice'], plot=plt)
plt.show()

[link numpy.log1p](https://docs.scipy.org/doc/numpy/reference/generated/numpy.log1p.html)

[link numpy.expm1](https://docs.scipy.org/doc/numpy/reference/generated/numpy.expm1.html#numpy.expm1)


In [0]:
price_train = np.log1p(np.array(train_data['SalePrice'].values))
n, bins, patches = plt.hist(price_train, 100, density=True, facecolor='g', alpha=0.75)
plt.show()

fig = plt.figure()
res = stats.probplot(np.log1p(train_data['SalePrice']), plot=plt)
plt.show()

In [0]:
def normalizer(data,is_one_dim = False):
  if (is_one_dim):
    min_max_scale = preprocessing.MinMaxScaler().fit(data.reshape((len(data),1)))
  else:
    min_max_scale = preprocessing.MinMaxScaler().fit(data)
  return min_max_scale

In [0]:
train_np = np.array(train_data[num_features].values)
print(np.where(np.isnan(train_np)))

In [0]:
train_np = np.array(train_data[num_features].values)
train_log1p_price = np.log1p(np.array(train_data['SalePrice'].values))
train_normalizer = normalizer(train_np)
train_np_norm = train_normalizer.transform(train_np)
price_normalizer = normalizer(train_log1p_price,is_one_dim = True)
train_log1p_price_norm = price_normalizer.transform(train_log1p_price.reshape(1,len(train_log1p_price))).reshape(len(train_log1p_price))

In [0]:
lin_reg = LinearRegression().fit(train_np_norm, train_log1p_price_norm)
train_predict_log1p_norm = lin_reg.predict(train_np_norm)
print(lin_reg.score(train_np_norm, train_log1p_price_norm))

train_predict_log1p = price_normalizer.inverse_transform(train_predict_log1p_norm.reshape((len(train_predict_log1p_norm),1))).reshape(len(train_predict_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(train_predict_log1p_norm,train_log1p_price_norm)) #mean_squared_error(train_predict_log1p_norm,train_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(train_predict_log1p)*delta_k)
print(delta_x)

In [0]:
val_np = np.array(val_data[num_features].values)
val_log1p_price = np.log1p(np.array(val_data['SalePrice'].values))
val_np_norm = train_normalizer.transform(val_np)
val_log1p_price_norm = price_normalizer.transform(val_log1p_price.reshape(1,len(val_log1p_price))).reshape(len(val_log1p_price))

print(lin_reg.score(val_np_norm, val_log1p_price_norm))

val_predict_log1p_norm = lin_reg.predict(val_np_norm)
val_predict_log1p = price_normalizer.inverse_transform(val_predict_log1p_norm.reshape((len(val_predict_log1p_norm),1))).reshape(len(val_predict_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(val_predict_log1p_norm,val_log1p_price_norm)) #mean_squared_error(val_predict_log1p_norm,val_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_
print(delta_k_norm)
print(delta_k)

delta_x = np.mean(np.exp(val_predict_log1p)*delta_k)
print(delta_x)

train test split

In [0]:
data_to_sel_feature = np.vstack((train_np,val_np))
log1p_price_all = np.hstack((train_log1p_price,val_log1p_price))

data_train = np.hstack((data_to_sel_feature,log1p_price_all.reshape((len(log1p_price_all),1))))
print(data_train.shape)

corre_mat = np.corrcoef(data_train.T)
ax = sns.heatmap(corre_mat, vmin=0, vmax=1)

In [0]:
print(num_features[3])
print(num_features[12])
print(num_features[13])
print(num_features[15])
print(num_features[25])
print(num_features[26])

## select feature
 What happens if you reduce the amount of features to just Sex and Age? (Use logistic regression)


In [0]:
train_cut_np = np.array(train_data[['OverallQual','1stFlrSF','2ndFlrSF','GrLivArea','GarageCars','GarageArea']].values)

train_cut_normalizer = normalizer(train_cut_np)

train_cut_norm = train_cut_normalizer.transform(train_cut_np)

lin_reg_cut = LinearRegression().fit(train_cut_norm, train_log1p_price_norm)
train_cut_predict_log1p_norm = lin_reg_cut.predict(train_cut_norm)
print(lin_reg_cut.score(train_cut_norm, train_log1p_price_norm))

train_cut_predict_log1p = price_normalizer.inverse_transform(train_cut_predict_log1p_norm.reshape((len(train_cut_predict_log1p_norm),1))).reshape(len(train_cut_predict_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(train_cut_predict_log1p_norm,train_log1p_price_norm)) #mean_squared_error(train_cut_predict_log1p_norm,train_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(train_cut_predict_log1p)*delta_k)
print(delta_x)

In [0]:
val_cut_np = np.array(val_data[['OverallQual','1stFlrSF','2ndFlrSF','GrLivArea','GarageCars','GarageArea']].values)

val_cut_norm = train_cut_normalizer.transform(val_cut_np)

val_cut_predict_log1p_norm = lin_reg_cut.predict(val_cut_norm)
print(lin_reg_cut.score(val_cut_norm, val_log1p_price_norm))

val_cut_predict_log1p = price_normalizer.inverse_transform(val_cut_predict_log1p_norm.reshape((len(val_cut_predict_log1p_norm),1))).reshape(len(val_cut_predict_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(val_cut_predict_log1p_norm,val_log1p_price_norm)) #mean_squared_error(val_cut_predict_log1p_norm,val_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(val_cut_predict_log1p)*delta_k)
print(delta_x)

## created complex feature

Try adding some higher order features to your training (x2 1,x1x2,...). 

[polynomial feature](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [0]:
train_poly_np = preprocessing.PolynomialFeatures(2).fit_transform(train_cut_np)

train_poly_normalizer = normalizer(train_poly_np)

train_poly_norm = train_poly_normalizer.transform(train_poly_np)

lin_reg_poly = LinearRegression().fit(train_poly_norm, train_log1p_price_norm)
train_poly_predict_log1p_norm = lin_reg_poly.predict(train_poly_norm)
print(lin_reg_poly.score(train_poly_norm, train_log1p_price_norm))

train_poly_predict_log1p = price_normalizer.inverse_transform(train_poly_predict_log1p_norm.reshape((len(train_poly_predict_log1p_norm),1))).reshape(len(train_poly_predict_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(train_poly_predict_log1p_norm,train_log1p_price_norm)) #mean_squared_error(train_poly_predict_log1p_norm,train_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(train_cut_predict_log1p)*delta_k)
print(delta_x)

In [0]:
val_poly_np = preprocessing.PolynomialFeatures(2).fit_transform(val_cut_np)

val_poly_norm = train_poly_normalizer.transform(val_poly_np)

val_poly_predict_log1p_norm = lin_reg_poly.predict(val_poly_norm)
print(lin_reg_poly.score(val_poly_norm, val_log1p_price_norm))

val_poly_predict_log1p = price_normalizer.inverse_transform(val_poly_predict_log1p_norm.reshape((len(val_poly_predict_log1p_norm),1))).reshape(len(val_poly_predict_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(val_poly_predict_log1p_norm,val_log1p_price_norm)) #mean_squared_error(val_poly_predict_log1p_norm,val_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(val_cut_predict_log1p)*delta_k)
print(delta_x)

[bar chart](https://pythonspot.com/matplotlib-bar-chart/)

In [0]:
coeff_np = np.array(lin_reg_poly.coef_)
ind = np.arange(coeff_np.shape[0])

plt.bar(ind, np.absolute(coeff_np), align='center', alpha=0.5)
plt.xticks(ind)
plt.ylabel('values')
plt.title('absolute of coefficient')
 
plt.show()

[link ElasticNet](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)

In [0]:
alpha_sel = 0.00001
l1_ratio_sel = 1.0

lin_reg_l1 = ElasticNet(alpha = alpha_sel, l1_ratio = l1_ratio_sel).fit(train_poly_norm, train_log1p_price_norm)
train_predict_l1_log1p_norm = lin_reg_l1.predict(train_poly_norm)
print(lin_reg_l1.score(train_poly_norm, train_log1p_price_norm))

train_predict_l1_log1p = price_normalizer.inverse_transform(train_predict_l1_log1p_norm.reshape((len(train_predict_l1_log1p_norm),1))).reshape(len(train_predict_l1_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(train_predict_l1_log1p_norm,train_log1p_price_norm)) #mean_squared_error(train_predict_l1_log1p_norm,train_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(train_predict_l1_log1p)*delta_k)
print(delta_x)

In [0]:
val_predict_l1_log1p_norm = lin_reg_l1.predict(val_poly_norm)
print(lin_reg_l1.score(val_poly_norm, val_log1p_price_norm))

val_predict_l1_log1p = price_normalizer.inverse_transform(val_predict_l1_log1p_norm.reshape((len(val_predict_l1_log1p_norm),1))).reshape(len(val_predict_l1_log1p_norm))

delta_k_norm = np.sqrt(mean_squared_error(val_predict_l1_log1p_norm,val_log1p_price_norm)) #mean_squared_error(val_predict_l1_log1p_norm,val_log1p_price_norm)
delta_k = delta_k_norm/price_normalizer.scale_

print(delta_k)
delta_x = np.mean(np.exp(val_predict_l1_log1p)*delta_k)
print(delta_x)

In [0]:
coeff_np = np.array(lin_reg_l1.coef_)
ind = np.arange(coeff_np.shape[0])

plt.bar(ind, np.absolute(coeff_np), align='center', alpha=0.5)
plt.xticks(ind)
plt.ylabel('values')
plt.title('absolute of coefficient')
 
plt.show()

## submit your result